#  DSCI 417 – Homework 08
**Lauren Forti**

In [0]:
# setup
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

spark = SparkSession.builder.getOrCreate()

## Problem 1: Creating Streaming DataFrame

In [0]:
# read file into static df
ps = (
    spark.read
    .option('delimiter', ',')
    .option('header', True)
    .option('inferSchema', True)
    .csv('/FileStore/tables/paysim/step_001.csv')
)

# store schema
ps_schema = ps.schema

# show df
ps.show(5, truncate=False)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+
step|type |amount |nameOrig |oldbalanceOrg|newbalanceOrig|nameDest |oldbalanceDest|newbalanceDest|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+
1 |PAYMENT |9839.64 |C1231006815|170136.0 |160296.36 |M1979787155|0.0 |0.0 |
1 |PAYMENT |1864.28 |C1666544295|21249.0 |19384.72 |M2044282225|0.0 |0.0 |
1 |TRANSFER|181.0 |C1305486145|181.0 |0.0 |C553264065 |0.0 |0.0 |
1 |CASH_OUT|181.0 |C840083671 |181.0 |0.0 |C38997010 |21182.0 |0.0 |
1 |PAYMENT |11668.14|C2048537720|41554.0 |29885.86 |M1230701703|0.0 |0.0 |
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+
only showing top 5 rows

In [0]:
# read file into streaming df
paysim_stream = (
  spark.readStream
  .option('header', True)
  .option('delimiter', ',')
  .option('maxFilesPerTrigger', 1)
  .schema(ps_schema)
  .csv('/FileStore/tables/paysim/')
)

print(paysim_stream.isStreaming)

True

## Problem 2: Apply Transformations

In [0]:
type_summary = (
  paysim_stream
  
  # group by type
  .groupBy('type')
  
  .agg(
    # count # of records
    expr('count(*) as n'),
    # get avg amount
    expr('round(mean(amount), 4) as avg_amount'),
    # get min amount
    expr('min(amount) as min_amount'),
    # get max amount
    expr('max(amount) as max_amount')
  )
  
  # sort by # of records desc
  .sort('n', ascending=False)
)

In [0]:
destinations = (
  paysim_stream
  
  # keep only transfer records
  .filter(col('type') == 'TRANSFER')
  
  # group by destination
  .groupBy('nameDest')
  .agg(
    expr('count(*) as n'),
    expr('sum(amount) as total_amount'),
    expr('round(mean(amount), 2) as avg_amount')
  )
  
  # sort by # of records desc
  .sort('n', ascending=False)
)

## Problem 3: Define Output Sinks

In [0]:
# creating data stream writer objects
t_writer = (
    type_summary
    .writeStream
    .format('memory')
    .queryName('type_summary')
    .outputMode('complete')
)

d_writer = (
    destinations
    .writeStream
    .format('memory')
    .queryName('destinations')
    .outputMode('complete')
)

## Problem 4: Start and Monitor the Streams

In [0]:
# start streams
t_query = t_writer.start()
d_query = d_writer.start()

In [0]:
# put contents of type_summary into static df
type_df = spark.sql('SELECT * from type_summary').toPandas()

# count # of observations
t_n = type_df.count()
# output
print(t_n)

# display df
type_df.display()

type 5
n 5
avg_amount 5
min_amount 5
max_amount 5
dtype: int64

type,n,avg_amount,min_amount,max_amount
CASH_OUT,250071,186314.0387,0.37,1.0E7
PAYMENT,234839,11587.4637,0.1,115264.68
CASH_IN,152356,171838.341,5.44,1289407.91
TRANSFER,57588,673235.3867,2.6,1.0E7
DEBIT,4971,5982.907,0.87,150053.28


In [0]:
# put contents of type_summary into static df
dest_df = spark.sql('SELECT * from destinations').toPandas()

# count # of observations
d_n = dest_df.count()
# output
print(d_n)

# display first 16 rows of df
dest_df.head(16)

nameDest 28668
n 28668
total_amount 28668
avg_amount 28668
dtype: int64
Out[40]:

,nameDest,n,total_amount,avg_amount
0,C665576141,26,26150103.90,1005773.23
1,C1286084959,26,17588259.10,676471.50
2,C248609774,24,24711862.80,1029660.95
3,C1360767589,22,21756362.92,988925.59
4,C306206744,22,13048885.07,593131.14
5,C985934102,21,13709050.24,652811.92
6,C1590550415,20,26350395.92,1317519.80
7,C451111351,20,19783480.34,989174.02
8,C97730845,20,28009878.86,1400493.94
9,C1883840933,19,23864765.21,1256040.27


In [0]:
# stop live streaming
t_query.stop()
d_query.stop()